# TensorRT Quantization Tutorial

This notebook is designed to show the features of the TensorRT passes integrated into MASE as part of the MASERT framework. The following demonstrations were run on a NVIDIA RTX A2000 GPU with a Intel(R) Xeon(R) CPU E5-2690 v4 @ 2.60GHz CPU.

## Section 1. INT8 Quantization
Firstly, we will show you how to do a int8 quantization of a simple model, `jsc-toy`, and compare the quantized model to the original model using the `Machop API`. The quantization process is split into the following stages, each using their own individual pass, and are explained in depth at each subsection:

1. [Fake quantization](#section-11-fake-quantization): `tensorrt_fake_quantize_transform_pass`
2. [Calibration](#section-12-calibration): `tensorrt_calibrate_transform_pass`
3. [Quantized Aware Training](#section-13-quantized-aware-training-qat): `tensorrt_fine_tune_transform_pass`
4. [Quantization](#section-14-tensorrt-quantization): `tensorrt_engine_interface_pass`
5. [Analysis](#section-15-performance-analysis): `tensorrt_analysis_pass`

We start by loading in the required libraries and passes required for the notebook as well as ensuring the correct path is set for machop to be used.

In [1]:
import sys
import os
from pathlib import Path
import toml

# Figure out the correct path
machop_path = Path(".").resolve().parent.parent.parent /"src"
assert machop_path.exists(), "Failed to find machop at: {}".format(machop_path)
sys.path.append(str(machop_path))

# Add directory to the PATH so that chop can be called
new_path = "../../../machop"
full_path = os.path.abspath(new_path)
os.environ['PATH'] += os.pathsep + full_path

import chop

from chop.tools.utils import to_numpy_if_tensor
from chop.tools.logger import set_logging_verbosity
from chop.tools import get_cf_args, get_dummy_input
from chop.passes.graph.utils import deepcopy_mase_graph
from chop.tools.get_input import InputGenerator
from chop.tools.checkpoint_load import load_model
from chop.ir import MaseGraph
from chop.models import get_model_info, get_model, get_tokenizer
from chop.dataset import MaseDataModule, get_dataset_info
from chop.passes.graph.transforms import metadata_value_type_cast_transform_pass
from chop.passes.graph import (
    summarize_quantization_analysis_pass,
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
    tensorrt_calibrate_transform_pass,
    tensorrt_fake_quantize_transform_pass,
    tensorrt_fine_tune_transform_pass,
    tensorrt_engine_interface_pass,
    runtime_analysis_pass,
    )

set_logging_verbosity("info")

/srcPkgs/miniconda3/lib/python3.11/site-packages/timm/models/helpers.py:7: FutureWarning: Importing from timm.models.helpers is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
INFO     Set logging level to info


Check dependency (the dependent package "cuda" refers to "cuda-python")

In [2]:
from chop.tools.check_dependency import check_deps_tensorRT_pass
check_deps_tensorRT_pass(silent=False)

INFO     Extension: All dependencies for TensorRT pass are available.


True

Next, we load in the toml file used for quantization. To view the configuration, click [here](../../../machop/configs/tensorrt/jsc_toy_INT8_quantization_by_type.toml).

In [3]:
import toml
# Path to your TOML file
# JSC_TOML_PATH = 'toy_INT8_quantization_by_type.toml'
JSC_TOML_PATH = 'resnet18_INT8_quantization_by_type.toml'

# Reading TOML file and converting it into a Python dictionary
with open(JSC_TOML_PATH, 'r') as toml_file:
    pass_args = toml.load(toml_file)

# Extract the 'passes.tensorrt' section and its children
tensorrt_config = pass_args.get('passes', {}).get('tensorrt', {})
print(tensorrt_config)
# Extract the 'passes.runtime_analysis' section and its children
runtime_analysis_config = pass_args.get('passes', {}).get('tensorrt', {}).get('runtime_analysis', {})
print(runtime_analysis_config)

{'by': 'type', 'num_calibration_batches': 10, 'post_calibration_analysis': True, 'default': {'config': {'quantize': True, 'calibrators': ['percentile', 'mse', 'entropy'], 'percentiles': [99.0, 99.9, 99.99], 'precision': 'int8'}, 'input': {'calibrator': 'histogram', 'quantize_axis': False}, 'weight': {'calibrator': 'histogram', 'quantize_axis': False}}, 'fine_tune': {'fine_tune': True}, 'runtime_analysis': {'num_batches': 500, 'num_GPU_warmup_batches': 5, 'test': True}}
{'num_batches': 500, 'num_GPU_warmup_batches': 5, 'test': True}


We then create a `MaseGraph` by loading in a model and training it using the toml configuration model arguments.

In [4]:
from chop.dataset import MaseDataModule
from chop.models import get_model_info
from chop.models import get_model
from chop.tools.get_input import InputGenerator

# Load the basics in
model_name = pass_args['model']
dataset_name = pass_args['dataset']
max_epochs = pass_args['max_epochs']
batch_size = pass_args['batch_size']
learning_rate = pass_args['learning_rate']
accelerator = pass_args['accelerator']

data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
)
data_module.prepare_data()
data_module.setup()

# Add the data_module and other necessary information to the configs
configs = [tensorrt_config, runtime_analysis_config]
for config in configs:
    config['task'] = pass_args['task']
    config['dataset'] = pass_args['dataset']
    config['batch_size'] = pass_args['batch_size']
    config['model'] = pass_args['model']
    config['data_module'] = data_module
    config['accelerator'] = 'cuda' if pass_args['accelerator'] == 'gpu' else pass_args['accelerator']
    if config['accelerator'] == 'gpu':
        os.environ['CUDA_MODULE_LOADING'] = 'LAZY'

model_info = get_model_info(model_name)
# quant_modules.initialize()
model = get_model(
    model_name,
    # task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False)


input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
)

# generate the mase graph and initialize node metadata
mg = MaseGraph(model=model)

model_info is MaseModelInfo(name='resnet', model_source=<ModelSource.TORCHVISION: 'torchvision'>, task_type=<ModelTaskType.VISION: 'vision'>, image_classification=True, physical_data_point_classification=False, sequence_classification=False, seq2seqLM=False, causal_LM=False, is_quantized=False, is_lora=False, is_sparse=False, is_fx_traceable=True)


Next, we train the `jsc-toy` model using the machop `train` action with the config from the toml file.

Run the following command to train a jsc-toy.

```bash
cd machop
./ch train --config ./configs/tensorrt/jsc_toy_INT8_quantization_by_type.toml
```

Then we load in the checkpoint. You will have to adjust this according to where it has been stored in the mase_output directory.

In [6]:
!python3 ./ch train --config /workspace/ADLS_Proj/docs/tutorials/tensorrt/resnet18_INT8_quantization_by_type.toml

/srcPkgs/miniconda3/lib/python3.11/site-packages/timm/models/helpers.py:7: FutureWarning: Importing from timm.models.helpers is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
INFO: Seed set to 0
I0308 00:19:22.817888 140004777665600 seed.py:57] Seed set to 0
+-------------------------+--------------------------+--------------+-----------------+--------------------------+
| Name                    |         Default          | Config. File | Manual Override |        Effective         |
+-------------------------+--------------------------+--------------+-----------------+--------------------------+
| task                    |      classification      |     cls      |                 |           cls            |
| load_name               |           None           |              |                 |           None           |
| load_type               |            mz            |           

In [7]:
!ls /workspace/ADLS_Proj/mase_output/*/software/training_ckpts/


/workspace/ADLS_Proj/mase_output/resnet18_cls_cifar10_2025-03-07/software/training_ckpts/:

/workspace/ADLS_Proj/mase_output/resnet18_cls_cifar10_2025-03-08/software/training_ckpts/:
best.ckpt  last.ckpt


In [12]:
# Load in the trained checkpoint - change this accordingly
JSC_CHECKPOINT_PATH = "/workspace/ADLS_Proj/mase_output/resnet18_cls_cifar10_2025-03-08/software/training_ckpts/best.ckpt"
mg = MaseGraph(model=model)

model = load_model(load_name=JSC_CHECKPOINT_PATH, load_type="pl", model=model)
print("load model done!")

# Initiate metadata
dummy_in = next(iter(input_generator))
print("dummy in done")

_ = model(**dummy_in)
print("_ done")

mg, _ = init_metadata_analysis_pass(mg, None)
print("init_metadata_analysis_pass done")

mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
print("add_common_metadata_analysis_pass done")

mg, _ = add_software_metadata_analysis_pass(mg, None)
print("add_software_metadata_analysis_pass done")

mg, _ = metadata_value_type_cast_transform_pass(mg, pass_args={"fn": to_numpy_if_tensor})
print("metadata_value_type_cast_transform_pass done")

# Before we begin, we will copy the original MaseGraph model to use for comparison during quantization analysis
#mg_original = deepcopy_mase_graph(mg)
#print("deep copy done")
from copy import deepcopy
import torch.fx as fx

fx_graph_copy = deepcopy(mg.fx_graph)

# 重新构造 MaseGraph
mg_original = MaseGraph(model=fx.GraphModule(mg.model, fx_graph_copy))

print("Deep copy of FX Graph and reconstruction of MaseGraph done")

INFO     Loaded pytorch lightning checkpoint from /workspace/ADLS_Proj/mase_output/resnet18_cls_cifar10_2025-03-08/software/training_ckpts/best.ckpt


load model done!
dummy in done
_ done
init_metadata_analysis_pass done
add_common_metadata_analysis_pass done
add_software_metadata_analysis_pass done
metadata_value_type_cast_transform_pass done
Deep copy of FX Graph and reconstruction of MaseGraph done


### Section 1.1 Fake Quantization

Firstly, we fake quantize the module in order to perform calibration and fine tuning before actually quantizing - this is only used if we have int8 calibration as other precisions are not currently supported within [pytorch-quantization](https://docs.nvidia.com/deeplearning/tensorrt/pytorch-quantization-toolkit/docs/index.html#) library.

This is acheived through the `tensorrt_fake_quantize_transform_pass` which goes through the model, either by type or by name, replaces each layer appropriately to a fake quantized form if the `quantize` parameter is set in the default config (`passes.tensorrt.default.config`) or on a per name or type basis. 

Currently the quantizable layers are:
- Linear
- Conv1d, Conv2d, Conv3d 
- ConvTranspose1d, ConvTranspose2d, ConvTranspose3d 
- MaxPool1d, MaxPool2d, MaxPool3d
- AvgPool1d, AvgPool2d, AvgPool3d
- LSTM, LSTMCell

To create a custom quantized module, click [here](https://docs.nvidia.com/deeplearning/tensorrt/pytorch-quantization-toolkit/docs/index.html#document-tutorials/creating_custom_quantized_modules).


In [13]:
mg, _ = tensorrt_fake_quantize_transform_pass(mg, pass_args=tensorrt_config)
# summarize_quantization_analysis_pass(mg_original, mg)
summarize_quantization_analysis_pass(mg, pass_args={"save_dir": "quantize_summary_res18", "original_graph": mg_original})

INFO     Applying fake quantization to PyTorch model...


op is placeholder
placeholder not in QUANTIZEABLE_OP
op is conv2d
node.op == call_module
op is batch_norm2d
batch_norm2d not in QUANTIZEABLE_OP
op is relu
relu not in QUANTIZEABLE_OP
op is max_pool2d
max_pool2d not in QUANTIZEABLE_OP
op is conv2d
node.op == call_module
op is batch_norm2d
batch_norm2d not in QUANTIZEABLE_OP
op is relu
relu not in QUANTIZEABLE_OP
op is conv2d
node.op == call_module
op is batch_norm2d
batch_norm2d not in QUANTIZEABLE_OP
op is add
add not in QUANTIZEABLE_OP
op is relu
relu not in QUANTIZEABLE_OP
op is conv2d
node.op == call_module
op is batch_norm2d
batch_norm2d not in QUANTIZEABLE_OP
op is relu
relu not in QUANTIZEABLE_OP
op is conv2d
node.op == call_module
op is batch_norm2d
batch_norm2d not in QUANTIZEABLE_OP
op is add
add not in QUANTIZEABLE_OP
op is relu
relu not in QUANTIZEABLE_OP
op is conv2d
node.op == call_module
op is batch_norm2d
batch_norm2d not in QUANTIZEABLE_OP
op is relu
relu not in QUANTIZEABLE_OP
op is conv2d
node.op == call_module
op is 

INFO     Fake quantization applied to PyTorch model.


op is batch_norm2d
batch_norm2d not in QUANTIZEABLE_OP
op is add
add not in QUANTIZEABLE_OP
op is relu
relu not in QUANTIZEABLE_OP
op is conv2d
node.op == call_module
op is batch_norm2d
batch_norm2d not in QUANTIZEABLE_OP
op is relu
relu not in QUANTIZEABLE_OP
op is conv2d
node.op == call_module
op is batch_norm2d
batch_norm2d not in QUANTIZEABLE_OP
op is add
add not in QUANTIZEABLE_OP
op is relu
relu not in QUANTIZEABLE_OP
op is adaptive_avg_pool2d
adaptive_avg_pool2d not in QUANTIZEABLE_OP
op is flatten
flatten not in QUANTIZEABLE_OP
op is linear
node.op == call_module
op is output
output not in QUANTIZEABLE_OP


INFO     Quantized graph histogram:
INFO     
| Original type     | OP                  |   Total |   Changed |   Unchanged |
|-------------------+---------------------+---------+-----------+-------------|
| AdaptiveAvgPool2d | adaptive_avg_pool2d |       1 |         0 |           1 |
| BatchNorm2d       | batch_norm2d        |      20 |         0 |          20 |
| MaxPool2d         | max_pool2d          |       1 |         0 |           1 |
| QuantConv2d       | conv2d              |      20 |         0 |          20 |
| QuantLinear       | linear              |       1 |         0 |           1 |
| ReLU              | relu                |      17 |         0 |          17 |
| add               | add                 |       8 |         0 |           8 |
| flatten           | flatten             |       1 |         0 |           1 |
| output            | output              |       1 |         0 |           1 |
| x                 | placeholder         |       1 |         0 |         

(<chop.ir.graph.mase_graph.MaseGraph at 0x7fb67c50e690>, {})

As you can see we have succesfully fake quantized all linear layers inside `jsc-toy`. This means that we will be able to simulate a quantized model in order to calibrate and fine tune it. This fake quantization was done on typewise i.e. for linear layers only. See [Section 4](#section-4-layer-wise-mixed-precision) for how to apply quantization layerwise - i.e. only first and second layers for example.

### Section 1.2 Calibration

Next, we perform calibration using the `tensorrt_calibrate_transform_pass`. Calibration is achieved by passing data samples to the quantizer and deciding the best amax for activations. 

Calibrators can be added as a search space parameter to examine the best performing calibrator. The calibrators have been included in the toml as follows.
For example: `calibrators = ["percentile", "mse", "entropy"]`

Note: 
- To use `percentile` calibration, a list of percentiles must be given
- To use `max` calibration, the `histogram` weight and input calibrators must be removed and replaced with `max`. This will use global maximum absolute value to calibrate the model.
- If `post_calibration_analysis` is set true the `tensorrt_analysis_pass` will be run for each calibrator tested to evaluate the most suitable calibrator for the model.

In [14]:
mg, _ = tensorrt_calibrate_transform_pass(mg, pass_args=tensorrt_config)

INFO     Starting calibration of the model in PyTorch...
INFO     Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization and Enabling Calibration
INFO     Disabling Quantization

From the results, the 99% `percentile` clips too many values during the amax calibration, compromising the loss. However 99.99% demonstrates higher validation accuracy alongside `mse` and `entropy` for `jsc-toy`. For such a small model, the methods are not highly distinguished, however for larger models this calibration process will be important for ensuring the quantized model still performs well. 

### Section 1.3 Quantized Aware Training (QAT)

The `tensorrt_fine_tune_transform_pass` is used to fine tune the quantized model. By default, when running the `tensorrt_engine_interface_pass` the fake quantized model will go through fine tuning however you stop this by setting the `fine_tune` in `passes.tensorrt.fine_tune` to false.

For QAT it is typical to employ 10% of the original training epochs, starting at 1% of the initial training learning rate, and a cosine annealing learning rate schedule that follows the decreasing half of a cosine period, down to 1% of the initial fine tuning learning rate (0.01% of the initial training learning rate). However this default can be overidden by setting the `epochs`, `initial_learning_rate` and `final_learning_rate` in `passes.tensorrt.fine_tune`.

The fine tuned checkpoints are stored in the ckpts/fine_tuning folder:

```
mase_output
└── tensorrt
    └── quantization
        └──model_task_dataset_date
            ├── cache
            ├── ckpts
            │   └── fine_tuning
            ├── json
            ├── onnx
            └── trt
```

In [15]:
mg, _ = tensorrt_fine_tune_transform_pass(mg, pass_args=tensorrt_config)

INFO     Starting Fine Tuning for 2 epochs...
I0308 01:31:58.107703 140431941403712 rank_zero.py:63] GPU available: True (cuda), used: True
I0308 01:31:58.108611 140431941403712 rank_zero.py:63] TPU available: False, using: 0 TPU cores
I0308 01:31:58.109225 140431941403712 rank_zero.py:63] HPU available: False, using: 0 HPUs
I0308 01:31:58.110737 140431941403712 rank_zero.py:63] You are using a CUDA device ('NVIDIA GeForce RTX 4060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
I0308 01:32:00.773122 140431941403712 cuda.py:61] LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
I0308 01:32:00.793271 140431941403712 model_summary.py:104] 
  | Name      | Type               | Params | Mode 
--------------------------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

I0308 01:35:01.178247 140431941403712 rank_zero.py:63] `Trainer.fit` stopped: `max_epochs=2` reached.
INFO     Fine Tuning Complete


### Section 1.4 TensorRT Quantization

After QAT, we are now ready to convert the model to a tensorRT engine so that it can be run with the superior inference speeds. To do so, we use the `tensorrt_engine_interface_pass` which converts the `MaseGraph`'s model from a Pytorch one to an ONNX format as an intermediate stage of the conversion.

During the conversion process, the `.onnx` and `.trt` files are stored to their respective folders shown in [Section 1.3](#section-13-quantized-aware-training-qat).

This interface pass returns a dictionary containing the `onnx_path` and `trt_engine_path`.

In [16]:
mg, meta = tensorrt_engine_interface_pass(mg, pass_args=tensorrt_config)

INFO     Converting PyTorch model to ONNX...
INFO     ONNX Conversion Complete. Stored ONNX model to /workspace/ADLS_Proj/mase_output/tensorrt/quantization/resnet18_cls_cifar10_2025-03-08/2025-03-08/version_4/model.onnx
INFO     Converting PyTorch model to TensorRT...
INFO     TensorRT Conversion Complete. Stored trt model to /workspace/ADLS_Proj/mase_output/tensorrt/quantization/resnet18_cls_cifar10_2025-03-08/2025-03-08/version_5/model.trt
INFO     TensorRT Model Summary Exported to /workspace/ADLS_Proj/mase_output/tensorrt/quantization/resnet18_cls_cifar10_2025-03-08/2025-03-08/version_6/model.json


### Section 1.5 Performance Analysis

To showcase the improved inference speeds and to evaluate accuracy and other performance metrics, the `tensorrt_analysis_pass` can be used.

The tensorRT engine path obtained the previous interface pass is now inputted into the the analysis pass. The same pass can take a MaseGraph as an input, as well as an ONNX graph. For this comparison, we will first run the anaylsis pass on the original unquantized model and then on the int8 quantized model.

In [17]:
_, _ = runtime_analysis_pass(mg_original, pass_args=runtime_analysis_config)
_, _ = runtime_analysis_pass(meta['trt_engine_path'], pass_args=runtime_analysis_config)

INFO     Starting transformation analysis on resnet18
INFO     
Results resnet18:
+------------------------------+--------------+
|      Metric (Per Batch)      |    Value     |
+------------------------------+--------------+
|    Average Test Accuracy     |   0.72852    |
|      Average Precision       |   0.71886    |
|        Average Recall        |   0.72227    |
|       Average F1 Score       |   0.71961    |
|         Average Loss         |   0.80973    |
|       Average Latency        |  7.6262 ms   |
|   Average GPU Power Usage    |   10.92 W    |
| Inference Energy Consumption | 0.023133 mWh |
+------------------------------+--------------+
INFO     Runtime analysis results saved to /workspace/mase_output/tensorrt/quantization/resnet18_cls_cifar10_2025-03-08/mase_graph/version_10/model.json
INFO     Starting transformation analysis on resnet18-trt_quantized


[03/08/2025-01:39:46] [TRT] [W] Using default stream in enqueueV3() may lead to performance issues due to additional calls to cudaStreamSynchronize() by TensorRT to ensure correct synchronization. Please use non-default stream instead.


INFO     
Results resnet18-trt_quantized:
+------------------------------+--------------+
|      Metric (Per Batch)      |    Value     |
+------------------------------+--------------+
|    Average Test Accuracy     |   0.74065    |
|      Average Precision       |   0.74343    |
|        Average Recall        |   0.74441    |
|       Average F1 Score       |   0.74278    |
|         Average Loss         |   0.73761    |
|       Average Latency        |  6.2613 ms   |
|   Average GPU Power Usage    |   10.897 W   |
| Inference Energy Consumption | 0.018952 mWh |
+------------------------------+--------------+
INFO     Runtime analysis results saved to /workspace/mase_output/tensorrt/quantization/resnet18_cls_cifar10_2025-03-08/tensorrt/version_0/model.json


As shown above, the latency has decreased around 6x with the `jsc-toy` model without compromising accuracy due to the well calibrated amax and quantization-aware fine tuning and additional runtime optimizations from TensorRT. The inference energy consumption has thus also dropped tremendously and this is an excellent demonstration for the need to quantize in industry especially for LLMs in order to reduce energy usage. 

## Section 2. FP16 Quantization

We will now load in a new toml configuration that uses fp16 instead of int8, whilst keeping the other settings the exact same for a fair comparison. This time however, we will use chop from the terminal which runs all the passes showcased in [Section 1](#section-1---int8-quantization).

Since float quantization does not require calibration, nor is it supported by `pytorch-quantization`, the model will not undergo fake quantization; for the time being this unfortunately means QAT is unavailable and only undergoes Post Training Quantization (PTQ). 

In [44]:
JSC_FP16_BY_TYPE_TOML = "/workspace/ADLS_Proj/docs/tutorials/tensorrt/resnet18_FP16_quantization_by_type.toml"
JSC_CHECKPOINT_PATH = "/workspace/ADLS_Proj/mase_output/resnet18_cls_cifar10_2025-03-08/software/training_ckpts/best.ckpt"
!python3 ch transform --config {JSC_FP16_BY_TYPE_TOML} --load {JSC_CHECKPOINT_PATH} --load-type pl

8351.89s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


/srcPkgs/miniconda3/lib/python3.11/site-packages/timm/models/helpers.py:7: FutureWarning: Importing from timm.models.helpers is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
INFO: Seed set to 0
I0308 03:44:00.043000 139717010794304 seed.py:57] Seed set to 0
+-------------------------+--------------------------+--------------+--------------------------+--------------------------+
| Name                    |         Default          | Config. File |     Manual Override      |        Effective         |
+-------------------------+--------------------------+--------------+--------------------------+--------------------------+
| task                    |      classification      |     cls      |                          |           cls            |
| load_name               |           None           |              | /workspace/ADLS_Proj/mas | /workspace/ADLS_Proj/mas |
|                   

As you can see, `fp16` acheives a slighty higher test accuracy but a slightly lower latency (~30%) from that of int8 quantization; it is still ~2.5x faster than the unquantized model. Now lets apply quantization to a more complicated model.

## Section 3. Type-wise Mixed Precision on Larger Model
We will now quantize `vgg7` which includes both convolutional and linear layers, however for this demonstration we want to quantize all layer types except the linear layers.

In this case, we set:

- The `by` parameter to `type`
- The `quantize` parameter to true for `passes.tensorrt.conv2d.config` and `precision` parameter to 'int8'.
- The `input` and `weight` quantize axis for the conv2d layers.
- The default `passes.tensorrt.default.config` precision to true. 

During the TensorRT quantization, the model's conv2d layers will be converted to an int8 fake quantized form, whilst the linear layers are kept to their default 'fp16'. Calibration of the conv2d layers and then fine tuning will be undergone before quantization and inference.

You may either download a pretrained model [here](https://imperiallondon-my.sharepoint.com/:f:/g/personal/zz7522_ic_ac_uk/Emh3VT7Q_qRFmnp8kDrcgDoBwGUuzLwwKNtX8ZAt368jJQ?e=gsKONa), otherwise train it yourself as shown below. 

In [22]:
VGG_TYPEWISE_TOML = "vgg7_typewise_mixed_precision.toml"

!python ch train --config {VGG_TYPEWISE_TOML}

/srcPkgs/miniconda3/lib/python3.11/site-packages/timm/models/helpers.py:7: FutureWarning: Importing from timm.models.helpers is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
INFO: Seed set to 0
I0308 01:49:21.999958 140686138111808 seed.py:57] Seed set to 0
+-------------------------+--------------------------+--------------+-----------------+--------------------------+
| Name                    |         Default          | Config. File | Manual Override |        Effective         |
+-------------------------+--------------------------+--------------+-----------------+--------------------------+
| task                    |      classification      |     cls      |                 |           cls            |
| load_name               |           None           |              |                 |           None           |
| load_type               |            mz            |           

We will now load the checkpoint in, quantize the model and compare it to the unquantized version as we did in [Section 1.5](#section-15-performance-analysis)

In [45]:
VGG_TYPEWISE_TOML = "vgg7_typewise_mixed_precision.toml"
# Change this checkpoint path accordingly
VGG_CHECKPOINT_PATH = "/workspace/ADLS_Proj/mase_output/vgg7_cifar_cls_cifar10_2025-03-08/software/training_ckpts/best.ckpt"

In [53]:
!python ch transform --config {VGG_TYPEWISE_TOML} --load {VGG_CHECKPOINT_PATH} --load-type pl

9993.82s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


/srcPkgs/miniconda3/lib/python3.11/site-packages/timm/models/helpers.py:7: FutureWarning: Importing from timm.models.helpers is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
INFO: Seed set to 0
I0308 04:11:18.312891 140245053151040 seed.py:57] Seed set to 0
+-------------------------+--------------------------+--------------+--------------------------+--------------------------+
| Name                    |         Default          | Config. File |     Manual Override      |        Effective         |
+-------------------------+--------------------------+--------------+--------------------------+--------------------------+
| task                    |      classification      |     cls      |                          |           cls            |
| load_name               |           None           |              | /workspace/ADLS_Proj/mas | /workspace/ADLS_Proj/mas |
|                   

By quantizing all convolutional layers to INT8 and maintaining fp16 precision for the linear layers we see a marginal decrease in latency whilst maintaining a comparable accuracy. By experimenting with precisions on a per type basis, you may find insights that work best for your model. 

## Section 4. Layer-wise Mixed Precision

So far we have strictly quantized either in int8 or fp16. Now, we will show how to conduct layerwise mixed precision using the same `vgg7` model. In this case we will show how for instance, layer 0 and 1 can be set to fp16, while the remaining layers can be int8 quantized. 

For this, we set:
- The `by` parameter to `name`
- The `precision` to 'int8' for `passes.tensorrt.default.config`
- The `precision` to 'fp16' for `passes.tensorrt.feature_layers_0.config and passes.tensorrt.feature_layers_1.config`
- The `precision` to 'int8' for `passes.tensorrt.feature_layers_2.config and passes.tensorrt.feature_layers_3.config` (although this is not necessary since the default is already set to 'int8')

In [67]:
VGG_LAYERWISE_TOML = "vgg7_layerwise_mixed_precision.toml"
VGG_CHECKPOINT_PATH = "/workspace/ADLS_Proj/mase_output/vgg7_cifar_cls_cifar10_2025-03-08/software/training_ckpts/best.ckpt"
!python ch transform --config {VGG_LAYERWISE_TOML} --load {VGG_CHECKPOINT_PATH} --load-type pl

12908.38s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


/srcPkgs/miniconda3/lib/python3.11/site-packages/timm/models/helpers.py:7: FutureWarning: Importing from timm.models.helpers is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
INFO: Seed set to 0
I0308 04:59:55.829589 139995367372608 seed.py:57] Seed set to 0
+-------------------------+--------------------------+--------------+--------------------------+--------------------------+
| Name                    |         Default          | Config. File |     Manual Override      |        Effective         |
+-------------------------+--------------------------+--------------+--------------------------+--------------------------+
| task                    |      classification      |     cls      |                          |           cls            |
| load_name               |           None           |              | /workspace/ADLS_Proj/mas | /workspace/ADLS_Proj/mas |
|                   

In this case, we can see through the quantized summary that one convolutional layer (feature_layers_1) has not been quantized as its precision will be configured to 'fp16' in the tensorrt engine conversion stage whilst the remaining convolutional and linear layers have been quantized.